In [ ]:
import pandas as pd

In [66]:
df1 = pd.read_csv('Parceria_USP_listagens.xlsx - Novas histórias.csv')
df2 = pd.read_csv('USP_projeto_ponto_cultura.xlsx - lista_personagem_projeto_ponto_.csv')

In [67]:
df1 = df1.rename(columns={"Link para a História": "Link Acervo", "Link para a íntegra": "Link Youtube"})

In [68]:
df2 = df2.rename(columns={"Vídeo no Acervo": "Link Acervo", "Video no Youtube": "Link Youtube"})

In [69]:
columns_to_keep = ['ID da História', 'Título da História',
       'Código de Acervo', 'Nome do Entrevistado', 'Biografia do Entrevistado',
       'Gênero', 'Data de Nascimento', 'País de Nascimento',
       'Estado de Nascimento', 'Cidade de Nascimento', 'Religião', 'Raça',
       'Etnia', 'Escolaridade', 'Profissão', 'Ocupação', 'Link Acervo','Link Youtube']

In [128]:
dff = pd.concat([df1[columns_to_keep], df2[columns_to_keep]], ignore_index=True).drop_duplicates(subset="Código de Acervo")

In [129]:
dff = dff.rename(columns={
    "ID da História": "mupe_id",
    "Título da História": "title",
    "Código de Acervo": "mupe_code",
    "Nome do Entrevistado": "interviewee_name",
    "Biografia do Entrevistado": "interviewee_bio",
    'Gênero': 'gender', 
    'Data de Nascimento': 'birthday', 
    'País de Nascimento': 'birth_country',
    'Estado de Nascimento': 'birth_state', 
    'Cidade de Nascimento':'birth_city', 
    'Religião':'religion', 
    'Raça': 'race',
    'Etnia': 'ethnicity', 
    'Escolaridade': 'education', 
    'Profissão': 'profession', 
    'Ocupação': 'occupation', 
    'Link Acervo': 'mupe_link',
    'Link Youtube': 'youtube_link'
})

In [76]:
dff.to_json('df.json', orient='records')

In [126]:
def remove_hashtag(x):
    if isinstance(x, str):
        return x.replace('#', '')
    elif isinstance(x, int):
        return int(str(x).replace('#', ''))
    else:
        return x

In [132]:
dff["mupe_id"] = dff["mupe_id"].apply(remove_hashtag)

In [143]:
from src.clients.database import Database

with Database() as db:
    for _, row in dff.iterrows():
        print(row['mupe_code'])
        print(row['interviewee_name'])
        normalized_name = row['interviewee_name'].replace("'", "''")

        query = f"""
SELECT *
FROM Audio
WHERE 
    json_metadata is null AND
    (name LIKE '%{row['mupe_code']}%' 
    OR 
    name LIKE '%{normalized_name}%'
    OR
    name LIKE '%{normalized_name.replace(' ', '_')}%')
LIMIT 1
"""
        # print(query)
        # break
        match = db._run_query(query)
        
        if isinstance(match, pd.DataFrame) and not match.empty:
            print(f"Match found! Updating audio {match.iloc[0]['name']}")
            json_string = row.to_json(orient='index')
            update_query = f"""
            UPDATE Audio
            SET json_metadata = %s
            WHERE id = {match.iloc[0]['id']}
            """
            print(json_string)
            db._run_query(update_query, (json_string,))
            print("\n==================\n\n")

    pass

MB_HV100
Antônio Raimundo de Souza Printes
MB_HV101
Manoel de Souza Lima
MB_HV102
Domingas Coimbra de Souza
MB_HV103
Maria Elba de Sousa Silva
MB_HV104
Pedro Santarém Marturano
MB_HV105
Bertino Gomes da Cruz
MB_HV106
Pedro Lima Silveira (São Pedro)
MB_HV107
Norbertino da Silva Pereira
MB_HV108
Sebastiana Gomes Pinheiro
MB_HV109
Aurecilia da Silva Andrade
MB_HV110
Parisio Silva
MB_HV111
Manoel Marialva da Silva
MB_HV112
Maria Sebastiana Alzier Marialva
MB_HV113
Raimundo Tavares Pimentel
MB_HV114
Zanira Torres dos Santos
MB_HV116
Sebastião Soares Serique
MB_HV118
Eleonora de Barros Hamoy
MB_HV119
Fortunato Chocron
MB_HV121
Domingas Serrão Pinto (Dona Domingas)
MB_HV122
Dulce Sardinha de Vasconcelos
MB_HV124
Jaime Ramos Marinho
MB_HV126
Pedro Chuck Cheeman Koapecck
MB_HV128
Amelia Lopes Legal
MB_HV129
Francisco Batista Vanziler (Seu Pará)
MB_HV130
Rosa Maria Lopes
MB_HV131
Maria Ninita da Silva
MB_HV132
Joaquim Gonçalves Mendes (Seu Belisca)
MB_HV133
Theodora Mariana da Silva Santos
MB_HV

In [146]:
import json
obj = json.load(open('data/export/mupe/PC_MA_HV002_Benedicta Gonçalves Pereira/PC_MA_HV002_Benedicta Gonçalves Pereira_metadata.json'))

In [151]:
obj

'{"race": null, "title": "Quer transformar o mundo? Vá lá e faça", "gender": "Feminino", "mupe_id": 8061, "birthday": "2/7/1937", "religion": null, "education": null, "ethnicity": null, "mupe_code": "PC_MA_HV002", "mupe_link": "https://museudapessoa.org/historia-detalhe/?id=8061", "birth_city": "ITARARÉ", "occupation": null, "profession": "Professora leiga (não formada)", "birth_state": "SÃO PAULO", "youtube_link": "https://www.youtube.com/watch?v=w3EyLFJaII8", "birth_country": "BRASIL", "interviewee_bio": "Benedicta Gonçalves Pereira nasceu em 07 de fevereiro de 1937. Com ascendência indígena por parte de pai e africana por parte da mãe, teve uma infância simples chegando a morar em uma cocheira no Jockey Club de São Paulo, onde seu pai trabalhava. Como não teve uma boa formação escolar, decide mais tarde fundar a sua própria escola. A Escola São José começou com 30 alunos e continua a educar até hoje as crianças para a vida.", "interviewee_name": "Benedicta Gonçalves Pereira"}'